# Popular Topics in Data Science

The objective of this project is to identify relevant and high-impact topics for a company that creates educational content in the field of data science.

To accomplish our mission, we will need access to current data about the topics the data science community is most interested in. Fortunately, [Stack Exchange](https://stackexchange.com/) hosts dedicated sites where users can collaborate and learn about a multitude of topics, including mathematics, physics and data science. Users can pose questions to the community, provide answers to other questions and discuss a wide range of topics. Users can vote on questions and answers, which makes it easy to identify informative posts.

Stack Exchange also maintains a public data base, which captures a wide variety of user activity. The [Stack Exchange Data Explorer (SEDE)](https://data.stackexchange.com/datascience/query/new) will allow us to write SQL queries to extract data, examine trends and formulate our insights to select the most relevant topics.

For this project, we will explore questions and answers posted on [Data Science Stack Exchange](https://datascience.stackexchange.com/) during 2022.


## Import Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Load Data

When it comes to current topics in the field of data science, the [SEDE](https://data.stackexchange.com/datascience/query/new) contains a trove of data. We will run some queries to explore the database and select the most relevant information.



### Tags

When a user submits a question to Stack Exchange, they can select up to five tags or keywords to categorize and describe the nature of the question. `Tags` will be an important attribute to consider as we identify popular topics in data science. 

Below is a query for the 15 most frequently used tags on Data Science Stack Exchange. By joining the `Tags` and `Posts` tables, we can also extract `Excerpts` or descriptions for proper tag usage. 


```SQL
SELECT TOP 15 
       t.Id,
       t.TagName,
       t.Count,
       REPLACE(REPLACE(p.Body, '<p>', ''), '</p>', '') AS Excerpt
  FROM Tags t
  JOIN Posts p ON p.Id = t.ExcerptPostId
 ORDER BY t.Count DESC;
```
 

The `tag_query_results.csv` file contains the results of this query.


Although we identified the most frequently used tags, we need more granular information to determine the most relevant topics. In a rapidly evolving field, such as data science, tags that were popular several years ago may no longer be relevant.


Below is a query for the 15 most frequently used tags on Data Science Stack Exchange. By joining the `Tags` and `Posts` tables, we can also extract `Excerpts` or descriptions for proper tag usage. 


```SQL
SELECT TOP 15 
       t.Id,
       t.TagName,
       t.Count,
       REPLACE(REPLACE(p.Body, '<p>', ''), '</p>', '') AS Excerpt
  FROM Tags t
  JOIN Posts p ON p.Id = t.ExcerptPostId
 ORDER BY t.Count DESC;
```
 

The `tag_query_results.csv` file contains the results of this query.


Although we identified the most frequently used tags, we need more granular information to determine the most relevant topics. In a rapidly evolving field, such as data science, tags that were popular several years ago may no longer be relevant.

In [2]:
tags = pd.read_csv("tag_query_results.csv")
print("15 Most Frequently Used Tags:")
print('\n')
print(tags.iloc[:,:-1]) # Excludes excerpt column
print('\n')
print("Description for machine-learning tag:")
print('\n')
print(tags.iloc[0,-1]) # Print Excerpt for first row

15 Most Frequently Used Tags:


     Id           TagName  Count
0     2  machine-learning  11051
1    46            python   6558
2   194     deep-learning   4775
3    81    neural-network   4280
4    77    classification   3180
5   324             keras   2728
6    47               nlp   2553
7   128      scikit-learn   2258
8   321        tensorflow   2183
9    72       time-series   1812
10  157        regression   1544
11   24                 r   1481
12   61           dataset   1438
13  557               cnn   1370
14   15        clustering   1359


Description for machine-learning tag:


Machine Learning is a subfield of computer science that draws on elements from algorithmic analysis, computational statistics, mathematics, optimization, etc. It is mainly concerned with the use of data to construct models that have high predictive/forecasting ability. Topics include modeling building, applications, theory, etc.



### Posts

There is alot of information in the Posts table. Lets write a query to examine the type of posts submitted to the Data Science Stack Exchange in 2022.

```SQL
SELECT PT.Id, 
       PT.Name,
       COUNT(*) AS PostCount
  FROM PostTypes AS PT
 INNER JOIN Posts AS P ON P.PostTypeId = PT.Id
 WHERE YEAR(P.CreationDate) = 2022
 GROUP BY PT.Id, PT.Name
 ORDER BY PT.Id ASC
 ```

The `post_types_2022.csv` file contains the results of this query, which are displayed in the table below. Here we can see that 5,299 questions and 3,673 answers were submitted in 2022.

In [ ]:
post_types = pd.read_csv("post_types_2022.csv")
print(post_types)

Lets take a closer look at...

Query of Top 3 Questions in 2022 by View Count:

```SQL
SELECT TOP 3 Id, PostTypeId, Title, ViewCount, Tags
FROM Posts
WHERE PostTypeId = 1 AND YEAR(CreationDate) = 2022
ORDER BY ViewCount DESC
```

Query of Top 3 Answers in 2022 by ViewCount:

```SQL
SELECT TOP 3 Id, PostTypeId, Title, ViewCount, Tags
FROM Posts
WHERE PostTypeId = 2 AND YEAR(CreationDate) = 2022
ORDER BY ViewCount DESC
```

Answers do not contain any information for:
- Title
- ViewCount
- Tags

Lets focus our analysis on questions posted to SEDE in 2022...

Now lets write a query to extract information about questions posted in 2022...

```SQL
SELECT ID, PostTypeId, CreationDate, Title, Tags, Score, 
       ViewCount, AnswerCount, FavoriteCount
  FROM Posts
  WHERE YEAR(CreationDate) = 2022 AND PostTypeId = 1
```
The `questions_2022.csv` file contains the results of this query.

In [ ]:
# Read csv file into a pandas dataframe, parse dates in CreationDate column

questions = pd.read_csv("questions_2022.csv",parse_dates=["CreationDate"])

In [ ]:
# Review summary of questions dataframe

questions.info()

# FavoriteCount is missing ALOT of values

In [ ]:
print(questions.head(3))

## Clean Data
- Title and Tags are classified as objects
- Tags are enclosed in angle brackets
- FavoriteCount is missing alot of data and classified as a float


In [ ]:
# Use a lambda function to see what types of objects are in Title column
questions["Title"].apply(lambda value: type(value)).unique()

In [ ]:
# Use a lambda function to see what types of objects are in Tags column
questions["Tags"].apply(lambda value: type(value)).unique()

In [ ]:
# Remove angle brackets from Tags
questions["Tags"] = questions["Tags"].str.replace("^<|>$", "",regex=True).str.split("><")
print(questions.head(3))



In [ ]:
# Fill in null values in FavoriteCount
questions.fillna(value={"FavoriteCount": 0}, inplace=True) 

# Convert FavoriteCount datatype to integer
questions["FavoriteCount"] = questions["FavoriteCount"].astype(int) 

questions.dtypes



Summarize data cleaning here...

## Popular Tags

Most used and most viewed tags...


### Frequently Used Tags

In [ ]:
# Create a dictionary to count the frequency of tag usage

tag_count = dict()

for tags in questions["Tags"]:
    for tag in tags:
        if tag in tag_count:
            tag_count[tag] += 1
        else:
            tag_count[tag] = 1

# Convert the dictionary to dataframe

tag_count = pd.DataFrame.from_dict(tag_count, orient="index")

tag_count

In [ ]:
# Rename column to Count
tag_count.rename(columns={0: "Count"}, inplace=True)

# Sort Values in Descending Order, Display Top 10
most_used = tag_count.sort_values(by="Count",ascending=False).head(10)
most_used

In [ ]:
# Plot a horizontal bar chart to visualize frequency of tag usage

most_used.plot(kind="barh")
plt.title("Frequently Used Tags in 2022")
plt.show()

### Frequently Viewed Tags

In [ ]:
# Create a dictionary to aggregate view counts for each tag 

tag_view_count = dict()

for index, row in questions.iterrows():
    for tag in row['Tags']:
        if tag in tag_view_count:
            tag_view_count[tag] += row['ViewCount']
        else:
            tag_view_count[tag] = row['ViewCount']

# Convert the dictionary to a dataframe
tag_view_count = pd.DataFrame.from_dict(tag_view_count, orient="index")

tag_view_count

In [ ]:
# Rename column to ViewCount
tag_view_count.rename(columns={0: "ViewCount"}, inplace=True)

# Sort Values in Descending Order, Display Top 10
most_viewed = tag_view_count.sort_values(by="ViewCount",ascending=False).head(10)
most_viewed


In [ ]:
# Plot a horizontal bar chart to visualize frequency of tag views

most_viewed.plot(kind="barh")
plt.title("Frequently Viewed Tags in 2022")
plt.show()

In [ ]:


fig, axes = plt.subplots(nrows=1, ncols=2)
fig.set_size_inches((24, 10))
plt.suptitle("Popular Tags in 2022")
most_used.plot(kind="barh", ax=axes[0], subplots=True)
axes[0].set_title("Frequently Used Tags")
most_viewed.plot(kind="barh", ax=axes[1], subplots=True)
axes[1].set_title("Frequently Viewed Tags")
plt.show()



### Related Tags

In [ ]:
# Create a list that contains all tags from tag_count dictionary

all_tags = list(tag_count.index)

# Create an empty dataframe, use the all_tags list to label rows and columns

co_occurence = pd.DataFrame(index=all_tags, columns=all_tags)
co_occurence.iloc[0:4,0:4]



In [ ]:
# Create a co-occurence matrix to examine tag relationships

co_occurence.fillna(0, inplace=True)

for tags in questions["Tags"]:
    co_occurence.loc[tags, tags] += 1

co_occurence.iloc[0:4,0:4]


In [ ]:
co_occurence_most_used = co_occurence.loc[most_used.index, most_used.index]

In [ ]:
co_occurence_most_used

In [ ]:
# Remove all instances where a tag co-occurs with itself

for i in range(co_occurence_most_used.shape[0]):
    co_occurence_most_used.iloc[i,i] = np.NaN



In [ ]:
# Create a heatmap of co-occurences with seaborn

sns.heatmap(co_occurence_most_used, cmap="Greens",annot=False)

## Tag Trends

```SQL
SELECT Id, CreationDate, Tags
  FROM Posts
 WHERE PostTypeId = 1;
```

In [ ]:
all_qs = pd.read_csv("tag_trends.csv", parse_dates=["CreationDate"])
print(all_qs.head(3))

all_qs["Tags"].apply(lambda value: type(value)).unique()


In [ ]:
# Clean angle brackets from the tags column

all_qs["Tags"] = all_qs["Tags"].str.replace("^<|>$", "",regex=True).str.split("><")

all_qs["Tags"]